In [54]:
import timm
import torch
from torch import nn
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [55]:
# 1. Загружаем EfficientNet
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("efficientnet_b4", pretrained=True)
model.classifier = nn.Identity()  # убираем классификатор
model = model.to(device)
model.eval()

model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

EfficientNet(
  (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (b

In [56]:
# 2. Препроцессинг
transform = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225])
])

In [57]:
def get_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = model(tensor).cpu().numpy().flatten()
    return emb

In [58]:
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, image_dir, transform):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = os.listdir(image_dir)
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        fname = self.image_files[idx]
        car_id = int(fname.split("_")[0])
        image = Image.open(os.path.join(self.image_dir, fname)).convert("RGB")
        tensor = self.transform(image)
        return tensor, car_id, fname

# -------------------------------
# 4. Функция для извлечения эмбеддингов с GPU батчами
# -------------------------------
def get_batch_embeddings(image_dir, batch_size=32):
    dataset = ImageDataset(image_dir, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    embeddings = {}
    model.to(device)
    model.eval()
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            tensors, car_ids, fnames = batch
            tensors = tensors.to(device)
            
            batch_embeddings = model(tensors)
            batch_embeddings = batch_embeddings.cpu().numpy()
            
            for i, car_id in enumerate(car_ids):
                car_id = car_id.item()
                if car_id not in embeddings:
                    embeddings[car_id] = []
                embeddings[car_id].append(batch_embeddings[i])
    
    return embeddings

# -------------------------------
# 5. Функция агрегации эмбеддингов
# -------------------------------
def aggregate_embeddings(embeddings_dict, method="mean+max", max_photos=4):
    """
    embeddings_dict: {car_id: [emb1, emb2, ...]}
    method: "mean", "max", "mean+max", "concat"
    """
    agg_emb = {}
    for car_id, emb_list in embeddings_dict.items():
        emb_array = np.stack(emb_list)
        if method == "mean":
            agg_emb[car_id] = np.mean(emb_array, axis=0)
        elif method == "max":
            agg_emb[car_id] = np.max(emb_array, axis=0)
        elif method == "mean+max":
            agg_emb[car_id] = np.concatenate([np.mean(emb_array, axis=0),
                                              np.max(emb_array, axis=0)])
        elif method == "concat":
            n_photos = emb_array.shape[0]
            if n_photos < max_photos:
                pad = np.zeros((max_photos - n_photos, emb_array.shape[1]))
                emb_array = np.vstack([emb_array, pad])
            agg_emb[car_id] = emb_array.flatten()
        else:
            raise ValueError("Unknown aggregation method")
    return agg_emb


# Использование
embeddings_train = get_batch_embeddings("train_images", batch_size=32)
embeddings_test = get_batch_embeddings("test_images", batch_size=32)

100%|██████████| 3065/3065 [04:34<00:00, 11.17it/s]


In [59]:
train_agg_emb = aggregate_embeddings(embeddings_train, method='mean')

In [60]:
# 5. В DataFrame
df_emb = pd.DataFrame.from_dict(train_agg_emb, orient="index")
df_emb.reset_index(inplace=True)
df_emb.rename(columns={"index": "ID"}, inplace=True)

# Теперь df_emb можно смержить с train_dataset по ID


In [61]:
embeds_train = df_emb.copy()

In [62]:
test_agg_emb = aggregate_embeddings(embeddings_test, method='mean')

In [63]:
# 5. В DataFrame
embeds_test = pd.DataFrame.from_dict(test_agg_emb, orient="index")
embeds_test.reset_index(inplace=True)
embeds_test.rename(columns={"index": "ID"}, inplace=True)

# Теперь df_emb можно смержить с train_dataset по ID

In [64]:
train = pd.read_csv('data/train.csv')
train.head()

,ID,equipment,body_type,drive_type,engine_type,doors_number,color,pts,audiosistema,diski,...,protivoygonnaya_sistema_mult_Иммобилайзер,protivoygonnaya_sistema_mult_Сигнализация,protivoygonnaya_sistema_mult_Спутник,protivoygonnaya_sistema_mult_Центральный замок,salon_mult_Кожаный руль,salon_mult_Люк,upravlenie_klimatom_mult_Атермальное остекление,upravlenie_klimatom_mult_Управление на руле,dist_nearest_city,nearest_city
0,58146,Базовая,Седан,Передний,Бензин,4,Синий,Дубликат,other,15.0,...,0,0,0,0,0,0,0,0,334.957449,Казань
1,112144,Базовая,Универсал,Задний,Бензин,5,Бежевый,Оригинал,other,14.0,...,0,1,0,0,0,0,0,0,315.525726,Волгоград
2,120705,other,Внедорожник,Полный,Гибрид,5,Чёрный,Электронный,other,15.0,...,1,1,1,1,1,1,1,1,622.473943,Волгоград
3,291392,Titanium,Седан,Передний,Бензин,4,Серебряный,Оригинал,6 колонок,16.0,...,0,1,0,1,1,0,0,1,19.805661,Нижний Новгород
4,35742,Базовая,Седан,Передний,Бензин,4,Чёрный,Оригинал,other,15.0,...,0,0,0,0,0,0,0,0,28.605091,Екатеринбург


In [65]:
train = train.merge(embeds_train, on='ID')

In [66]:
cat_features = ['equipment', 'body_type', 'drive_type', 'engine_type', 'color', 'pts', 'audiosistema', 'electropodemniki', 'fary', 'salon', 'upravlenie_klimatom',
       'usilitel_rul', 'steering_wheel', 'nearest_city'
       # 'crashes_count', 'owners_count'
       ]

In [68]:
from catboost import CatBoostRegressor, Pool

In [69]:
# Данные
X_train = train.drop(columns=['price_TARGET'])
y_train = train['price_TARGET']

train_pool = Pool(X_train, y_train, cat_features=cat_features)

In [70]:
# Модель
model = CatBoostRegressor(
    iterations=3000,
    learning_rate=0.05,
    depth=8,
    loss_function='MAE',  # medianAPE похож на MAE по логам
    eval_metric='MAPE',
    l2_leaf_reg=7,
    bagging_temperature=1.0,
    random_seed=185,
    early_stopping_rounds=50,
    verbose=100, 
    task_type='GPU',
    border_count=254
)

model.fit(train_pool)

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 0.0626988	total: 85.1ms	remaining: 4m 15s
100:	learn: 0.0285877	total: 8.11s	remaining: 3m 52s
200:	learn: 0.0252711	total: 16.1s	remaining: 3m 44s
300:	learn: 0.0237073	total: 24s	remaining: 3m 34s
400:	learn: 0.0225501	total: 31.9s	remaining: 3m 26s
500:	learn: 0.0215914	total: 40s	remaining: 3m 19s
600:	learn: 0.0207408	total: 48.1s	remaining: 3m 11s
700:	learn: 0.0199284	total: 55.9s	remaining: 3m 3s
800:	learn: 0.0192092	total: 1m 4s	remaining: 2m 56s
900:	learn: 0.0185226	total: 1m 12s	remaining: 2m 47s
1000:	learn: 0.0178717	total: 1m 19s	remaining: 2m 39s
1100:	learn: 0.0172631	total: 1m 27s	remaining: 2m 31s
1200:	learn: 0.0166841	total: 1m 35s	remaining: 2m 23s
1300:	learn: 0.0161296	total: 1m 43s	remaining: 2m 15s
1400:	learn: 0.0155877	total: 1m 51s	remaining: 2m 7s
1500:	learn: 0.0150760	total: 1m 59s	remaining: 1m 59s
1600:	learn: 0.0145888	total: 2m 7s	remaining: 1m 51s
1700:	learn: 0.0141223	total: 2m 15s	remaining: 1m 43s
1800:	learn: 0.0136709	total: 2m 23s	

In [71]:
test = pd.read_csv('data/test.csv')
test.head()

,ID,equipment,body_type,drive_type,engine_type,doors_number,color,pts,audiosistema,diski,...,protivoygonnaya_sistema_mult_Иммобилайзер,protivoygonnaya_sistema_mult_Сигнализация,protivoygonnaya_sistema_mult_Спутник,protivoygonnaya_sistema_mult_Центральный замок,salon_mult_Кожаный руль,salon_mult_Люк,upravlenie_klimatom_mult_Атермальное остекление,upravlenie_klimatom_mult_Управление на руле,dist_nearest_city,nearest_city
0,8,Базовая,Седан,Задний,Бензин,2,Жёлтый,other,other,15.0,...,0,0,0,0,0,0,0,0,407.607911,Екатеринбург
1,26,Базовая,Хетчбэк,Передний,Бензин,3,Белый,other,other,15.0,...,0,0,0,0,0,0,0,0,32.505023,Челябинск
2,33,other,Внедорожник,Полный,Дизель,5,Синий,other,6 колонок,19.0,...,0,1,0,1,1,0,0,0,32.354804,Санкт-Петербург
3,52,Sport,Седан,Передний,Бензин,4,Серебряный,Оригинал,8+ колонок,17.0,...,0,1,0,1,1,0,1,1,827.314855,Санкт-Петербург
4,119,Базовая,Хетчбэк,Передний,Бензин,3,Синий,other,other,15.0,...,0,1,0,1,1,0,0,0,816.818496,Санкт-Петербург


In [72]:
test = test.merge(embeds_test, on='ID')

In [73]:
preds = model.predict(test)

In [74]:
preds = np.expm1(preds)

In [75]:
test['target'] = preds

In [76]:
test[['ID', 'target']].to_csv('subs/embeds3.csv', index=False)